# AdStackr
| Test Notebook |
| ---: |
| Feb. 24 20026 |
| by Quante Carlo |

In [19]:
import requests
import json
import random
from urllib.parse import urlparse, parse_qs

#BASE_URL = "http://localhost:5000"
BASE_URL = "http://ec2-3-148-173-179.us-east-2.compute.amazonaws.com:8000"

def generate_creatives(line_item_id, template_id, variants):
    url = f"{BASE_URL}/adstacker/generate_creatives"
    payload = {
        "dv360_line_item_id": line_item_id,
        "template_id": template_id,
        "variants": variants,
    }
    resp = requests.post(url, json=payload)
    resp.raise_for_status()
    data = resp.json()
    print("[generate_creatives]", line_item_id)
    print(json.dumps(data, indent=2))
    return data

def dv360_bid_and_win(segment):
    bid_url = f"{BASE_URL}/dv360/bid_request"
    bid_resp = requests.post(bid_url, json={"segment": segment})
    bid_resp.raise_for_status()
    bid_data = bid_resp.json()
    auction_id = bid_data["auction_id"]

    win_url = f"{BASE_URL}/adstacker/handle_win/{auction_id}"
    win_resp = requests.post(win_url)
    win_resp.raise_for_status()
    win_data = win_resp.json()
    print(f"[win] segment={segment}")
    print(json.dumps(win_data, indent=2))
    return win_data

def fire_click(click_url):
    """
    Simulate user click: call the click tracker URL and follow redirect.
    """
    resp = requests.get(click_url, allow_redirects=False)
    print("[click] status", resp.status_code, "Location:", resp.headers.get("Location"))
    return resp

def fire_conversion(creative_id):
    """
    Simulate a conversion pixel firing on vendor site.
    """
    url = f"{BASE_URL}/track/conversion"
    resp = requests.get(url, params={"creative_id": creative_id})
    print("[conversion] creative_id", creative_id, "status", resp.status_code)
    return resp


def get_metrics(creative_id=None, line_item_id=None):
    url = f"{BASE_URL}/metrics"
    params = {}
    if creative_id:
        params["creative_id"] = creative_id
    if line_item_id:
        params["line_item_id"] = line_item_id
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    print("[metrics]")
    print(json.dumps(data, indent=2))
    return data


1) Create creatives for two segments / line items

In [21]:
generate_creatives(
    line_item_id="dv_li_001",
    template_id="tmpl_hero_banner",
    variants=[
        {"variant_id": "a1", "headline": "SegA Headline", "bg_color": "#FFAAAA",
         "final_url": "https://example.com/segA"},
    ],
)

[generate_creatives] dv_li_001
{
  "creatives": [
    {
      "click_url": "http://ec2-3-148-173-179.us-east-2.compute.amazonaws.com:8000/track/click?creative_id=cm_creative_dv_li_001_a1&line_item_id=dv_li_001&template_id=tmpl_hero_banner&dest=https%3A%2F%2Fexample.com%2FsegA",
      "creative_id": "cm_creative_dv_li_001_a1",
      "data": {
        "bg_color": "#FFAAAA",
        "final_url": "https://example.com/segA",
        "headline": "SegA Headline",
        "variant_id": "a1"
      },
      "line_item_id": "dv_li_001"
    }
  ],
  "status": "creatives_set"
}


{'creatives': [{'click_url': 'http://ec2-3-148-173-179.us-east-2.compute.amazonaws.com:8000/track/click?creative_id=cm_creative_dv_li_001_a1&line_item_id=dv_li_001&template_id=tmpl_hero_banner&dest=https%3A%2F%2Fexample.com%2FsegA',
   'creative_id': 'cm_creative_dv_li_001_a1',
   'data': {'bg_color': '#FFAAAA',
    'final_url': 'https://example.com/segA',
    'headline': 'SegA Headline',
    'variant_id': 'a1'},
   'line_item_id': 'dv_li_001'}],
 'status': 'creatives_set'}

In [9]:

generate_creatives(
    line_item_id="dv_li_002",
    template_id="tmpl_hero_banner",
    variants=[
        {"variant_id": "b1", "headline": "SegB Headline", "bg_color": "#AAAFFF",
         "final_url": "https://example.com/segB"},
    ],
)



2) Generate a bunch of impressions, with random clicks/conversions

In [24]:
impressions = []
for i in range(50):
    seg = "segment_A" if random.random() < 0.5 else "segment_B"
    win = dv360_bid_and_win(seg)
    impressions.append(win)

    # 20% of impressions click
    if win["chosen_creative_id"] and random.random() < 0.2:
        click_url = win["click_url"]
        fire_click(click_url)

        # 50% of clickers convert
        if random.random() < 0.5:
            fire_conversion(win["chosen_creative_id"])



[win] segment=segment_A
{
  "auction_id": "db2b8463-3518-47be-9619-3f792f748442",
  "chosen_creative_id": "cm_creative_dv_li_001_a1",
  "click_url": "http://ec2-3-148-173-179.us-east-2.compute.amazonaws.com:8000/track/click?creative_id=cm_creative_dv_li_001_a1&line_item_id=dv_li_001&template_id=tmpl_hero_banner&dest=https%3A%2F%2Fexample.com%2FsegA",
  "line_item_id": "dv_li_001",
  "segment": "segment_A"
}
[win] segment=segment_B
{
  "auction_id": "244a25c4-e417-4ef8-9b35-a6788f0a53a2",
  "chosen_creative_id": null,
  "click_url": null,
  "line_item_id": "dv_li_002",
  "segment": "segment_B"
}
[win] segment=segment_B
{
  "auction_id": "39384926-39eb-401b-bf40-814d5a488468",
  "chosen_creative_id": null,
  "click_url": null,
  "line_item_id": "dv_li_002",
  "segment": "segment_B"
}
[win] segment=segment_B
{
  "auction_id": "50d85e72-b60a-41f6-970c-bcaa7cd1fa19",
  "chosen_creative_id": null,
  "click_url": null,
  "line_item_id": "dv_li_002",
  "segment": "segment_B"
}
[win] segment=se

3) Inspect metrics

In [25]:

get_metrics()  # all creatives
get_metrics(line_item_id="dv_li_001")
get_metrics(line_item_id="dv_li_002")


[metrics]
{
  "cm_creative_dv_li_001_a1": {
    "clicks": 5,
    "conversions": 3,
    "creative_id": "cm_creative_dv_li_001_a1",
    "impressions": 40,
    "line_item_id": "dv_li_001"
  }
}
[metrics]
{
  "cm_creative_dv_li_001_a1": {
    "clicks": 5,
    "conversions": 3,
    "creative_id": "cm_creative_dv_li_001_a1",
    "impressions": 40,
    "line_item_id": "dv_li_001"
  }
}
[metrics]
{}


{}

In [17]:

def generate_creatives_old(line_item_id, template_id, variants):
    """
    Call /adstacker/generate_creatives for a given line item.
    """
    url = f"{BASE_URL}/adstacker/generate_creatives"
    payload = {
        "dv360_line_item_id": line_item_id,
        "template_id": template_id,
        "variants": variants,
    }
    resp = requests.post(url, json=payload)
    resp.raise_for_status()
    data = resp.json()
    print("[generate_creatives]", line_item_id, resp.status_code)
    print(json.dumps(data, indent=2))
    return data

def dv360_bid_and_win_old(segment):
    """
    Simulate DV360 bid request + AdStac.kr win handling for a segment.
    """
    # Bid request
    bid_url = f"{BASE_URL}/dv360/bid_request"
    bid_resp = requests.post(bid_url, json={"segment": segment})
    bid_resp.raise_for_status()
    bid_data = bid_resp.json()
    auction_id = bid_data["auction_id"]
    print(f"[bid_request] segment={segment} auction_id={auction_id}")

    # Handle win
    win_url = f"{BASE_URL}/adstacker/handle_win/{auction_id}"
    win_resp = requests.post(win_url)
    win_resp.raise_for_status()
    win_data = win_resp.json()
    print("[handle_win] response:")
    print(json.dumps(win_data, indent=2))
    return win_data






In [26]:
dv360_bid_and_win("segment_A")
dv360_bid_and_win("segment_B")

[win] segment=segment_A
{
  "auction_id": "e11946b4-ce7b-4ead-a0c2-807eef01b8cb",
  "chosen_creative_id": "cm_creative_dv_li_001_a1",
  "click_url": "http://ec2-3-148-173-179.us-east-2.compute.amazonaws.com:8000/track/click?creative_id=cm_creative_dv_li_001_a1&line_item_id=dv_li_001&template_id=tmpl_hero_banner&dest=https%3A%2F%2Fexample.com%2FsegA",
  "line_item_id": "dv_li_001",
  "segment": "segment_A"
}
[win] segment=segment_B
{
  "auction_id": "7bdde0a6-6473-4fb0-9779-4afd40975286",
  "chosen_creative_id": null,
  "click_url": null,
  "line_item_id": "dv_li_002",
  "segment": "segment_B"
}


{'auction_id': '7bdde0a6-6473-4fb0-9779-4afd40975286',
 'chosen_creative_id': None,
 'click_url': None,
 'line_item_id': 'dv_li_002',
 'segment': 'segment_B'}